### Imports

In [1]:
import torch
import torch.nn as nn
import torch.functional as F
from data.dataset import (
    cifar10_trainloader,
    ciaf10_testloader,
    cifar100_trainloader,
    ciaf100_testloader,
)
import pandas as pd
import numpy as np
from copy import deepcopy
from torch.utils.data import Subset
import random
from pruning.structured_pruning import *

c:\Users\Fatim_Sproj\anaconda3\envs\bacp\lib\site-packages\torch\cuda\__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


### Cifar10 model and loader

In [2]:
train10 = cifar10_trainloader()
test10= ciaf10_testloader()
train100 = cifar100_trainloader()
test100 =  ciaf100_testloader()

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg16_bn", pretrained=True)
original_model = deepcopy(model)
model.to(device)

train_loader = cifar10_trainloader()
test_loader = ciaf10_testloader()

random.seed(42)
calib_indices = random.sample(range(len(train_loader.dataset)), 512)
calib_subset = Subset(train_loader.dataset, calib_indices)
calib_loader = torch.utils.data.DataLoader(calib_subset, batch_size=128)

plan_file_path = r"C:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\aiedge\Pruning\results\sensitivity_layerwise_cifar10.csv"
sparsity_plan = generate_plan_for_target_sparsity(plan_file_path, original_model, 0.73)

print("\nUsing Sparsity Plan:")
print(pd.Series(sparsity_plan).to_string())

Using cache found in C:\Users\Fatim_Sproj/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master


Files already downloaded and verified
Files already downloaded and verified

Using Sparsity Plan:
features.0     64.644968
features.3     64.644968
features.7     64.644968
features.10    64.644968
features.14    64.644968
features.17    64.644968
features.20    64.644968
features.24    64.644968
features.27    64.644968
features.30    64.644968
features.34    64.644968
features.37    64.644968
features.40    64.644968


In [10]:
print("\nAnalyzing original model to get stable channel rankings...")
original_activations_in, original_activations_out = collect_inputs_outputs(original_model, calib_loader, device)
original_unf_x = unfold_activations(original_activations_in, original_model)
channel_saliency_scores = get_channel_saliency_scores(original_model, original_unf_x, original_activations_out)

prunable_convs = [name for name, mod in model.named_modules() if isinstance(mod, nn.Conv2d)][1:]

for layer_name in prunable_convs:
    print(f"\nProcessing layer: {layer_name}")
    
    current_layer = model.get_submodule(layer_name)
    sparsity = sparsity_plan.get(layer_name, 0.0) / 100.0
    num_channels_to_keep = max(16, int(current_layer.in_channels * (1.0 - sparsity)))
    
    print(f"Target: Keep {num_channels_to_keep} / {current_layer.in_channels} channels")
    
    saliency = channel_saliency_scores[layer_name]
    kept_indices = np.argsort(-saliency)[:num_channels_to_keep]
    
    reconstruct_and_prune_prev(model, original_unf_x, original_activations_out, layer_name, kept_indices, device)
    
print("\nPruning complete.")

final_conv_channels = None
for layer in reversed(model.features):
    if isinstance(layer, nn.BatchNorm2d):
        final_conv_channels = layer.num_features
        break

if final_conv_channels:
    model.classifier[0] = nn.Linear(final_conv_channels, model.classifier[0].out_features)
    model.to(device)
else:
    print("Error: Could not find final BatchNorm layer to resize classifier.")


Analyzing original model to get stable channel rankings...


c:\Users\Fatim_Sproj\anaconda3\envs\bacp\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.593e+00, tolerance: 2.472e+00
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\Fatim_Sproj\anaconda3\envs\bacp\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.766e+00, tolerance: 4.422e-01
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\Fatim_Sproj\anaconda3\envs\bacp\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider i


Processing layer: features.3
Target: Keep 22 / 64 channels

Processing layer: features.7
Target: Keep 22 / 64 channels

Processing layer: features.10
Target: Keep 45 / 128 channels

Processing layer: features.14
Target: Keep 45 / 128 channels

Processing layer: features.17
Target: Keep 90 / 256 channels

Processing layer: features.20
Target: Keep 90 / 256 channels

Processing layer: features.24
Target: Keep 90 / 256 channels

Processing layer: features.27
Target: Keep 181 / 512 channels

Processing layer: features.30
Target: Keep 181 / 512 channels

Processing layer: features.34
Target: Keep 181 / 512 channels

Processing layer: features.37
Target: Keep 181 / 512 channels

Processing layer: features.40
Target: Keep 181 / 512 channels

Pruning complete.


In [12]:
sparsity = calculate_structured_sparsity(original_model, model)
print(f"\nOverall Structured Sparsity: {sparsity*100:.2f}%")
pre_finetune_acc = evaluate_accuracy(model, test_loader, device)
print(f"Accuracy after pruning: {pre_finetune_acc:.2f}%")

print("\nStarting fine-tuning...")
finetuned_model = finetune_model(model, train_loader, test_loader, device)

print("\n--- Final Results ---")
final_accuracy = evaluate_accuracy(finetuned_model, test_loader, device)
print(f"Final Accuracy: {final_accuracy:.2f}%")
print(f"Overall Sparsity: {sparsity*100:.2f}%")


Overall Structured Sparsity: 80.92%
Accuracy after pruning: 13.05%

Starting fine-tuning...
Epoch 1/40 — Accuracy: 78.54%
Epoch 2/40 — Accuracy: 78.97%
Epoch 3/40 — Accuracy: 78.39%
Epoch 4/40 — Accuracy: 79.64%
Epoch 5/40 — Accuracy: 80.90%
Epoch 6/40 — Accuracy: 82.58%
Epoch 7/40 — Accuracy: 82.37%
Epoch 8/40 — Accuracy: 83.88%
Epoch 9/40 — Accuracy: 83.88%
Epoch 10/40 — Accuracy: 84.51%
Epoch 11/40 — Accuracy: 84.20%
Epoch 12/40 — Accuracy: 82.09%
Epoch 13/40 — Accuracy: 83.89%
Epoch 14/40 — Accuracy: 85.65%
Epoch 15/40 — Accuracy: 85.18%
Epoch 16/40 — Accuracy: 85.09%
Epoch 17/40 — Accuracy: 83.22%
Epoch 18/40 — Accuracy: 85.32%
Epoch 19/40 — Accuracy: 84.30%
Epoch 20/40 — Accuracy: 85.21%
Epoch 21/40 — Accuracy: 85.58%
Epoch 22/40 — Accuracy: 85.09%
Epoch 23/40 — Accuracy: 86.70%
Epoch 24/40 — Accuracy: 86.49%
Epoch 25/40 — Accuracy: 86.62%
Epoch 26/40 — Accuracy: 87.23%
Epoch 27/40 — Accuracy: 87.54%
Epoch 28/40 — Accuracy: 87.45%
Epoch 29/40 — Accuracy: 87.71%
Epoch 30/40 — Acc

In [13]:
sparsity_int = int(sparsity * 100)
save_path = f"cifar10_vgg16_pruned_{sparsity_int}sparsity.pt"
torch.save(finetuned_model, save_path)
print(f"\nModel weights saved successfully to: {save_path}")


Model weights saved successfully to: cifar10_vgg16_pruned_80sparsity.pt


### Cifar100

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_vgg16_bn", pretrained=True)
original_model = deepcopy(model)
model.to(device)

train_loader = cifar100_trainloader()
test_loader = ciaf100_testloader()

random.seed(42)
calib_indices = random.sample(range(len(train_loader.dataset)), 512)
calib_subset = Subset(train_loader.dataset, calib_indices)
calib_loader = torch.utils.data.DataLoader(calib_subset, batch_size=128)

plan_file_path = r"C:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\aiedge\Pruning\results\sensitivity_layerwise_cifar100.csv"
sparsity_plan = generate_plan_for_target_sparsity(plan_file_path, original_model, 0.73)

print("\nUsing Sparsity Plan:")
print(pd.Series(sparsity_plan).to_string())

Using cache found in C:\Users\Fatim_Sproj/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master


Files already downloaded and verified
Files already downloaded and verified

Using Sparsity Plan:
features.0     64.644968
features.3     64.644968
features.7     64.644968
features.10    64.644968
features.14    64.644968
features.17    64.644968
features.20    64.644968
features.24    64.644968
features.27    64.644968
features.30    64.644968
features.34    64.644968
features.37    64.644968
features.40    64.644968


In [16]:
print("\nAnalyzing original model to get stable channel rankings...")
original_activations_in, original_activations_out = collect_inputs_outputs(original_model, calib_loader, device)
original_unf_x = unfold_activations(original_activations_in, original_model)
channel_saliency_scores = get_channel_saliency_scores(original_model, original_unf_x, original_activations_out)

prunable_convs = [name for name, mod in model.named_modules() if isinstance(mod, nn.Conv2d)][1:]

for layer_name in prunable_convs:
    print(f"\nProcessing layer: {layer_name}")
    
    current_layer = model.get_submodule(layer_name)
    sparsity = sparsity_plan.get(layer_name, 0.0) / 100.0
    num_channels_to_keep = max(16, int(current_layer.in_channels * (1.0 - sparsity)))
    
    print(f"Target: Keep {num_channels_to_keep} / {current_layer.in_channels} channels")
    
    saliency = channel_saliency_scores[layer_name]
    kept_indices = np.argsort(-saliency)[:num_channels_to_keep]
    
    reconstruct_and_prune_prev(model, original_unf_x, original_activations_out, layer_name, kept_indices, device)
    
print("\nPruning complete.")

final_conv_channels = None
for layer in reversed(model.features):
    if isinstance(layer, nn.BatchNorm2d):
        final_conv_channels = layer.num_features
        break

if final_conv_channels:
    model.classifier[0] = nn.Linear(final_conv_channels, model.classifier[0].out_features)
    model.to(device)
else:
    print("Error: Could not find final BatchNorm layer to resize classifier.")


Analyzing original model to get stable channel rankings...


c:\Users\Fatim_Sproj\anaconda3\envs\bacp\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.251e+01, tolerance: 1.200e+01
  model = cd_fast.enet_coordinate_descent_gram(



Processing layer: features.3
Target: Keep 22 / 64 channels

Processing layer: features.7
Target: Keep 22 / 64 channels

Processing layer: features.10
Target: Keep 45 / 128 channels

Processing layer: features.14
Target: Keep 45 / 128 channels

Processing layer: features.17
Target: Keep 90 / 256 channels

Processing layer: features.20
Target: Keep 90 / 256 channels

Processing layer: features.24
Target: Keep 90 / 256 channels

Processing layer: features.27
Target: Keep 181 / 512 channels

Processing layer: features.30
Target: Keep 181 / 512 channels

Processing layer: features.34
Target: Keep 181 / 512 channels

Processing layer: features.37
Target: Keep 181 / 512 channels

Processing layer: features.40
Target: Keep 181 / 512 channels

Pruning complete.


In [17]:
sparsity = calculate_structured_sparsity(original_model, model)
print(f"\nOverall Structured Sparsity: {sparsity*100:.2f}%")
pre_finetune_acc = evaluate_accuracy(model, test_loader, device)
print(f"Accuracy after pruning: {pre_finetune_acc:.2f}%")

print("\nStarting fine-tuning...")
finetuned_model = finetune_model(model, train_loader, test_loader, device)

print("\n--- Final Results ---")
final_accuracy = evaluate_accuracy(finetuned_model, test_loader, device)
print(f"Final Accuracy: {final_accuracy:.2f}%")
print(f"Overall Sparsity: {sparsity*100:.2f}%")


Overall Structured Sparsity: 80.68%
Accuracy after pruning: 1.18%

Starting fine-tuning...
Epoch 1/40 — Accuracy: 9.43%
Epoch 2/40 — Accuracy: 20.31%
Epoch 3/40 — Accuracy: 30.13%
Epoch 4/40 — Accuracy: 35.61%
Epoch 5/40 — Accuracy: 38.98%
Epoch 6/40 — Accuracy: 41.36%
Epoch 7/40 — Accuracy: 43.50%
Epoch 8/40 — Accuracy: 44.31%
Epoch 9/40 — Accuracy: 44.89%
Epoch 10/40 — Accuracy: 45.53%
Epoch 11/40 — Accuracy: 47.34%
Epoch 12/40 — Accuracy: 49.61%
Epoch 13/40 — Accuracy: 47.12%
Epoch 14/40 — Accuracy: 47.95%
Epoch 15/40 — Accuracy: 49.98%
Epoch 16/40 — Accuracy: 48.00%
Epoch 17/40 — Accuracy: 50.19%
Epoch 18/40 — Accuracy: 50.99%
Epoch 19/40 — Accuracy: 49.68%
Epoch 20/40 — Accuracy: 50.78%
Epoch 21/40 — Accuracy: 50.14%
Epoch 22/40 — Accuracy: 51.83%
Epoch 23/40 — Accuracy: 51.82%
Epoch 24/40 — Accuracy: 52.28%
Epoch 25/40 — Accuracy: 51.94%
Epoch 26/40 — Accuracy: 52.00%
Epoch 27/40 — Accuracy: 52.72%
Epoch 28/40 — Accuracy: 53.38%
Epoch 29/40 — Accuracy: 53.07%
Epoch 30/40 — Accur

In [18]:
sparsity_int = int(sparsity * 100)
save_path = f"cifar100_vgg16_pruned_{sparsity_int}sparsity.pt"
torch.save(finetuned_model, save_path)
print(f"\nModel weights saved successfully to: {save_path}")


Model weights saved successfully to: cifar100_vgg16_pruned_80sparsity.pt
